Vamos a diseñar un enfoque para entrenar un modelo único usando tanto el dataset tabular como las imágenes. Dado que tu dataset es desbalanceado y contiene valores faltantes, es crucial preparar adecuadamente los datos antes de entrenar el modelo. Aquí te explico cómo puedes proceder.

1. Preprocesamiento de Datos

1.1. Preprocesamiento del Dataset Tabular

	1.	Manejo de Valores Faltantes:
	•	Categorías Missing: Para columnas categóricas con valores faltantes (como sex, age_approx, anatom_site_general), puedes reemplazar los valores faltantes con una categoría especial como “missing”. Esto se hace para mantener la información de que estos datos estaban ausentes.
	•	Codificación: Utiliza OneHotEncoder o pd.get_dummies para convertir las variables categóricas en variables dummy. Asegúrate de incluir la categoría “missing” como una opción.
	2.	Escalado de Datos:
	•	Las variables numéricas pueden necesitar escalado. Utiliza StandardScaler o MinMaxScaler para normalizar los datos.
	3.	Balanceo del Dataset:
	•	Dado el desbalance en tu objetivo (target), puedes considerar técnicas de balanceo como SMOTE o undersampling para equilibrar las clases.
	4.	División del Dataset:
	•	Divide el dataset en conjuntos de entrenamiento y validación.

In [2]:
#lirerias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical



In [12]:
# Cargar el DataFrame 
df = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/dataset_16_20.csv')

# Asegurarse de que la columna 'image_id' tenga la extensión '.jpg'
df['image_id'] = df['image_id'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')

# Convertir la columna target a entero
df['target'] = df['target'].astype(int)

# Dividir el DataFrame en conjunto de entrenamiento y validación
df_train, df_val = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)

# Separar características y etiquetas
X_train = df_train[['image_id', 'sex', 'age_approx', 'anatom_site_general']]
y_train = df_train['target']

X_val = df_val[['image_id', 'sex', 'age_approx', 'anatom_site_general']]
y_val = df_val['target']

df

,image_id,sex,age_approx,anatom_site_general,target
0,16_ISIC_0000001.jpg,NaN,NaN,NaN,0
1,16_ISIC_0000002.jpg,NaN,NaN,NaN,1
2,16_ISIC_0000004.jpg,NaN,NaN,NaN,1
3,16_ISIC_0000006.jpg,NaN,NaN,NaN,0
4,16_ISIC_0000007.jpg,NaN,NaN,NaN,0
...,...,...,...,...,...
71365,20_ISIC_9999134.jpg,male,50.0,torso,0
71366,20_ISIC_9999320.jpg,male,65.0,torso,0
71367,20_ISIC_9999515.jpg,male,20.0,lower extremity,0
71368,20_ISIC_9999666.jpg,male,50.0,lower extremity,0


In [4]:
import tensorflow as tf
import numpy as np

def preprocess_image(image):
    image = tf.image.resize(image, [100, 100])
    image = image / 255.0  # Normalizar
    return image

# Crear un Dataset para las imágenes
def image_generator(df, directory, batch_size):
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        images = []
        labels = []
        for _, row in batch_df.iterrows():
            image_path = f'{directory}/{row["image_id"]}'
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = preprocess_image(image)
            images.append(image)
            labels.append(row["target"])
        yield np.array(images), np.array(labels)

# Crear datasets para entrenamiento y validación
train_dataset = tf.data.Dataset.from_generator(
    lambda: image_generator(df_train, '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image', 32),
    output_signature=(
        tf.TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).shuffle(1000).batch(32)

val_dataset = tf.data.Dataset.from_generator(
    lambda: image_generator(df_val, '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image', 32),
    output_signature=(
        tf.TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).batch(32)

2024-08-25 19:52:41.779488: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-08-25 19:52:41.779528: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-25 19:52:41.779537: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-25 19:52:41.779832: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-25 19:52:41.779875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
def preprocess_tabular(df):
    # Convertir datos tabulares a numpy arrays
    X = df[['sex', 'age_approx', 'anatom_site_general']].values
    return X

# Procesar datos tabulares para entrenamiento y validación
X_train_tabular = preprocess_tabular(df_train)
X_val_tabular = preprocess_tabular(df_val)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

df_train['target'] = df_train['target'].astype(str)
df_val['target'] = df_val['target'].astype(str)

# Crear generador de imágenes
def image_generator(df, image_directory, batch_size):
    image_datagen = ImageDataGenerator(rescale=1.0/255)
    return image_datagen.flow_from_dataframe(
        dataframe=df,
        directory=image_directory,
        x_col='image_id',
        y_col='target',
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )

# Preprocesar datos tabulares
def preprocess_tabular(df):
    # Aquí puedes aplicar cualquier preprocesamiento necesario
    # Ejemplo: convertir a numpy array y normalizar
    return df.to_numpy()

# Crear un generador combinado para imágenes y datos tabulares
def combined_generator(image_gen, df_tabular):
    def generator():
        for images, labels in image_gen:
            # Obtener índices del batch de imágenes
            indices = np.arange(len(images))
            tabular_data = preprocess_tabular(df_tabular.iloc[indices])
            yield [images, tabular_data], labels
    return generator

# Crear generadores para entrenamiento y validación
train_image_gen = image_generator(df_train, '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image', 32)
val_image_gen = image_generator(df_val, '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image', 32)

train_gen = combined_generator(train_image_gen, df_train)
val_gen = combined_generator(val_image_gen, df_val)

Found 57096 validated image filenames belonging to 2 classes.
Found 14274 validated image filenames belonging to 2 classes.


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate

# Definir las entradas
image_input = Input(shape=(100, 100, 3))
tabular_input = Input(shape=(100))

# Definir el modelo de imagen
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Concatenar con los datos tabulares
x = Concatenate()([x, tabular_input])
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Crear el modelo
model = Model(inputs=[image_input, tabular_input], outputs=output)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

# Definir el callback EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear
    patience=5,          # Número de épocas sin mejora para detener el entrenamiento
    restore_best_weights=True  # Restaurar los mejores pesos
)

In [11]:
# Entrenar el modelo
history = model.fit(
    x=train_gen(),
    validation_data=val_gen(),
    epochs=15,
    callbacks=[early_stop]
)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).